In [1]:
import os, sys
sys.path.append( '..' )
numGenes = 50
rootdir = '/gstore/data/tmai/Yasin/TMONC/HE2OMICS/TRL/2024_1_8/'
num_split = 10
num_fold = 5

In [2]:
import argparse
import pickle as pkl
import pandas as pd
import numpy as np
from collections import Counter
#from wsi_data import subset_wsi_dataset

import torch
from torch import nn, optim
import torch.multiprocessing as mp
from torch.utils.data import Subset

import copy as cp
import yaml
from sklearn.model_selection import KFold, StratifiedKFold

import matplotlib.pyplot as plt
import configparser

In [3]:
from experiment import Experiment
from copy import deepcopy
#sortedFile = 'results_single_split_sorted.csv'

In [4]:
from wsi_data import patient_stratified_kfold,patient_stratified_single_run
from omic_materials.model import OmicToSubtype
from omic_materials.experiment import OmicSetup
from omic_materials.train import *

In [5]:
def subset_wsi_dataset(dataset, subset_ix):
    """ Subset WSI dataset class instance with indices from subset_ix
    """
    dataset_copy = deepcopy(dataset)
    dataset_copy.samples = [dataset.samples[ix] for ix in subset_ix]
    dataset_copy.patients = np.array([dataset.patients[ix] for ix in subset_ix])
    dataset_copy.trials = pd.Series([dataset.trials[ix] for ix in subset_ix])
    dataset_copy.subtypes = pd.Series([dataset.subtypes[ix] for ix in subset_ix])
    return(dataset_copy)  

In [6]:
def convert_to_subtype_prediction(dataset,class_names,scaleFeatures=True,n_tiles=False):
    """
    Change (image,expression) tuples to (image,subtype), and make AlltilesWSI dataset
    """
    # string names for subtypes
    subtype = [str(i) for i in list(group2_dataset.subtypes)]
    # convert to indices
    subtype = [class_names.index(name) for name in subtype]
    
    dataset_copy = deepcopy(dataset)
    new_samples = [(image,int(subtype[idx])) for idx,(image,_) in enumerate(dataset_copy.samples)]
    #dataset_copy.samples = new_samples
    subtype_dataset = AlltilesWSI(new_samples,scaleFeatures=scaleFeatures,n_tiles=n_tiles)
    return(subtype_dataset)

### Define config files, use 50 most discriminant genes in HE2Omics

In [7]:
confi = yaml.load(open(os.path.join(rootdir, "config_main_"+str(numGenes)+"genes.yaml"), "r"), Loader=yaml.FullLoader)
print('Using configuration defined in {}'.format(confi['config']))

Using configuration defined in /gstore/data/tmai/Yasin/TMONC/HE2OMICS/TRL/2024_1_8/configs/IMvigor_3trial_50genes.ini


### Build dataset and permute patients stratifying on trial and subtype

In [8]:
config = confi['config']
exp = Experiment(config)

# Read architecture and training parameters
model_params = exp._read_architecture()
training_params = exp._read_training_params()

# Build dataset
dataset = exp._build_dataset()
# UNIQUE PATIENT IDENTIFIERS
pat_ids_unique, uniq_indices = np.unique(dataset.patients,return_index=True)
TOT_pat = pat_ids_unique.shape[0]
print(f'Number of patients: {TOT_pat}')

# STRATIFY BY TRIAL AND SUBTYPE
class_list = []
for trial,subtype in zip(dataset.trials,dataset.subtypes):
    class_list.append(trial+'_NMF'+str(subtype))
class_list = np.array(class_list)
class_list_patients = class_list[uniq_indices]

# PARAMETERS
Nsplit = 10
kfold_group1 = 5

# PERMUTE PATIENTS
ind_testing = np.zeros((Nsplit, TOT_pat))
np.random.seed(42)
for i in range(Nsplit):
    ind_testing[i, :] = np.random.permutation(TOT_pat)
ind_testing = ind_testing.astype(int)

logTransform: False, scaleVariables: False, ensureNonnegativity: True
Number of variables in input: 51
Number of genes found in transcriptome data: 51
Filtered out 161 slides less than 1000 tiles
number of images with paired omic data: 2285
Number of patients: 1810


### Prep omics to subtype config file

In [9]:
# Create folder for transfer learning output
trl_dir = exp.savedir + '_TRL'
os.makedirs(trl_dir, exist_ok=True)

# Create folder to store AUC and ACC results
outfolder = 'results' 
os.makedirs(os.path.join(trl_dir,outfolder), exist_ok=True)

config_path = os.path.join(rootdir,'omic_materials/omic_config_4000_tiles.ini')
print('Using configuration defined in {}'.format(config_path))
omic_setup = OmicSetup(config_path) 

model_params = omic_setup._read_architecture()
training_params = omic_setup._read_training_params()
build_params = omic_setup._build_feature_data()

model_params['n_classes'] = 4
#model_params['input_feature_size'] depends on scores, so will be determined inside loop

if model_params['p_dropout_fc'] is not None:
    model_params['dropout_fc'] = True
    
if model_params['p_dropout_atn'] is not None:
    model_params['dropout_atn'] = True
    
scaleOmicFeatures = False
if build_params['scaleOmicFeatures'] == "Yes":
    scaleOmicFeatures = True
    
n_tiles = False
if isinstance(build_params['num_tiles'], int):
    n_tiles = build_params['num_tiles']
print(n_tiles)

class_names = ['1','2','3','4']
#kfold_group2 = 5

do1 = model_params['p_dropout_fc']
do2 = model_params['p_dropout_atn']
me = training_params['max_epochs']
pa = training_params['patience']
sc = build_params['scaleOmicFeatures']
nt = build_params['num_tiles']
### OPtimization and scheduling
my_config = configparser.ConfigParser()
my_config.read(config_path)
dic1 = my_config['optimization']
lr = dic1['lr']
wd = dic1['weight_decay']
dic2 = my_config['scheduler']
ss = dic2['step_size']
gam = dic2['gamma']
### Print all
outfile_base = f"OMIC_{model_params['model_size']}_do_{do1}_{do2}_epoch_{me}_pat_{pa}_scale_{sc}_tile_{nt}_lr_{lr}_wd_{wd}_step_{ss}_gamma_{gam}"
print(outfile_base)

Using configuration defined in /gstore/data/tmai/Yasin/TMONC/HE2OMICS/TRL/2024_1_8/omic_materials/omic_config_4000_tiles.ini
4000
OMIC_nano_do_0.2_0.2_epoch_200_pat_30_scale_No_tile_4000_lr_1e-4_wd_0.1_step_10_gamma_0.9


### Obtain HE2Omics embeddings and do transfer learning

In [ ]:
# loop from 0 to Nsplit

start_split = 6
start_fold = 0
Ncval = 5
ml_strategy = ['train','validation']

#for rs in range(start_split,Nsplit):
for rs in range(start_split,start_split+1):
    
    # Creating a folder for each split
    split_savedir = os.path.join(trl_dir,'split'+str(rs))
    os.makedirs(split_savedir, exist_ok=True)
        
    ind = ind_testing[rs, :]
    shuffled_patient_classes = class_list_patients[ind]
    shuffled_patients = pat_ids_unique[ind]

    skf = StratifiedKFold(n_splits=2, shuffle=False) # shuffling is done earlier with permutations

    # Transcriptomic learning gets n_splits-1 folds (index1), classifier training gets 1 fold (index 2)
    # index1, index2 = next(iter(skf.split(ind, shuffled_patient_classes)))
    group1_index, group2_index = next(iter(skf.split(ind, shuffled_patient_classes)))

    print(f'Number of group 1 patients: {len(group1_index)}, Number of group 2 patients: {len(group2_index)}')

    pat_G1 = shuffled_patients[group1_index]
    pat_G2 = shuffled_patients[group2_index]
    pat_G1_classes = shuffled_patient_classes[group1_index]
    pat_G2_classes = shuffled_patient_classes[group2_index]

    # Make group 1 and group 2 datasets
    slide_ix_group1 = [ix for ix, d in enumerate(dataset.patients) if d in pat_G1]
    slide_ix_group2 = [ix for ix, d in enumerate(dataset.patients) if d in pat_G2]
    group1_dataset = subset_wsi_dataset(dataset,slide_ix_group1)
    
    group2_dataset = subset_wsi_dataset(dataset,slide_ix_group2)
    subtypes = list(group2_dataset.subtypes)

    print(f'Number of group 1 images: {len(group1_dataset)}, Number of group 2 images: {len(group2_dataset)}')
    

    for fold in range(start_fold,num_fold):
        
        #if rs==start_split and fold < start_fold:
        #    continue
        print(f'Running split {str(rs)}, fold {str(fold)}')

        ################################################################################ 
        # Obtain HE2Omics embeddings and make omic data
        ################################################################################ 
        model = torch.load(os.path.join('./' + exp.savedir,'split'+str(rs),'fold'+str(fold)+'_transcriptome_model.pt'),map_location='cpu')

        scores = []
        for count, sample in enumerate(group2_dataset.samples):
            #if count % 300 == 0:
            #    print(count)

            HE = np.load(sample[0])
            x = torch.Tensor(HE[np.newaxis].transpose(1, 2, 0))
            model.eval()
            with torch.no_grad():
                scores.append(model.conv_spare_last(x).detach().numpy().squeeze()) # Run partial inference

        # Each element in the 'scores' list object is able to inform about input features size
        model_params['input_feature_size'] = scores[0].shape[1]

        ### Define omic dataset
        omicdata = AlltilesOmic(scores,subtypes,class_names,scaleOmicFeatures=scaleOmicFeatures,n_tiles=n_tiles)

        ################################################################################
        # Creating CLAM folds (and a folder for each OMIC fold)
        ################################################################################
        fold_savedir = os.path.join(split_savedir,'fold'+str(fold))
        os.makedirs(fold_savedir, exist_ok=True)
        
        ## Stratified cross-validation in CLAM, stratify by subtype and trial
        # Different random seed for each OMIC fold to create CLAM folds
        #cval = fold   
        #train_idx, valid_idx, test_idx = patient_stratified_kfold(
        #                group2_dataset, n_splits=kfold_group2, valid_size=0.2,
        #                random_state=12*cval,stratifySubtype=False) # stratify set to False because it did not work when only 1/6 of the samples are used to train classifier

        ### Outfile name - base extensions
        out_overall = os.path.join(trl_dir,outfolder,outfile_base + f"_split_{rs}_fold_{fold}_overall.csv")
        out_auc = os.path.join(trl_dir,outfolder,outfile_base + f"_split_{rs}_fold_{fold}_class_AUC.csv")
        out_acc = os.path.join(trl_dir,outfolder,outfile_base + f"_split_{rs}_fold_{fold}_class_ACC.csv")
        # Text file
        outname = os.path.join(trl_dir,outfolder,outfile_base + f"_split_{rs}_fold_{fold}_output.txt")

        ov_metric = np.zeros((Ncval,2 * len(ml_strategy)))
        auc_collect = np.zeros((len(ml_strategy) * Ncval,len(class_names)))
        acc_collect = np.zeros((len(ml_strategy) * Ncval,len(class_names)))
        

        for cval in range(Ncval):  
            print(f'split: {rs}, OMIC fold:{fold}, CLAM fold: {cval}',file=open(outname, 'a'))
            print(f'split: {rs}, OMIC fold:{fold}, CLAM fold: {cval}')
            
            train_idx, valid_idx = patient_stratified_single_run(group2_dataset, valid_size=0.2, 
                                                                 stratifySubtype=True,random_state=12*cval)

            #################### save current folds
            this = (group2_dataset.patients[train_idx],group2_dataset.patients[valid_idx])
            with open(os.path.join(fold_savedir, 'group2_fold_patient_ids_TRL_clamFold' + str(cval) + '.pkl'), 'wb') as handle:
                pkl.dump(this, handle)
            ################################################################################ 

            # Separate train, validation, and test sets  
            train_set = Subset(omicdata, train_idx)
            valid_set = Subset(omicdata, valid_idx)

            ##### Omics-based classifier
            omic_model = OmicToSubtype(**model_params) # attention model
            omic_optimizer = omic_setup._setup_optimization(omic_model)
            #scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=4,gamma=0.9)
            omic_scheduler = omic_setup._setup_scheduler(omic_optimizer)

            # Train
            print("Training omics-based classifier")
            metric, class_auc, class_acc = omic_fit(omic_model,train_set,valid_set,
                               params=training_params,optimizer=omic_optimizer,
                               scheduler=omic_scheduler,
                               path=os.path.join(fold_savedir,'CLAM_fold_'+str(cval)),
                               outname = outname)
            
            ov_metric[cval] = metric.values
            # Allow printing output at each inner loop iteration
            ind_name = ['omicfold' + str(fold) + '_clamfold' + str(jnk) for jnk in range(Ncval)]
            ov_metric_df = pd.DataFrame(ov_metric,columns = ['tr_auc','tr_acc','val_auc','val_acc'],
                            index = ind_name)
            ov_metric_df.to_csv(out_overall,index=True)

            auc_collect[( len(ml_strategy)*cval):( len(ml_strategy) *(cval+1))] = class_auc
            # Allow printing output at each inner loop iteration
            auc_df = pd.DataFrame(auc_collect, columns = ['class1','class2','class3','class4'],
                        index = np.tile(['train','val'],Ncval))

            acc_collect[( len(ml_strategy)*cval):( len(ml_strategy) *(cval+1))] = class_acc
            # Allow printing output at each inner loop iteration
            acc_df = pd.DataFrame(acc_collect, columns = ['class1','class2','class3','class4'],
                        index = np.tile(['train','val'],Ncval))

            auc_df.to_csv(out_auc,index=True)
            acc_df.to_csv(out_acc,index=True)

Number of group 1 patients: 905, Number of group 2 patients: 905
Number of group 1 images: 1153, Number of group 2 images: 1132
Running split 6, fold 0
split: 6, OMIC fold:0, CLAM fold: 0
Training omics-based classifier


  0%|          | 0/920 [00:00<?, ?it/s]

Val set AUROC: 0.4643, accuracy: 0.189
Num epochs since best: 1


  0%|          | 0/920 [00:00<?, ?it/s]

train loss: 1.1523, val loss: 0.9976
Num epochs since best: 1


...

train loss: 0.7248, val loss: 0.9805
Num epochs since best: 29


  0%|          | 0/899 [00:00<?, ?it/s]

train loss: 0.7295, val loss: 0.9853
Num epochs since best: 30


100%|██████████| 899/899 [00:08<00:00, 109.91it/s]


train loss: 0.7295, val loss: 0.9547
Running split 6, fold 1


In [ ]:
start_split = 5
start_fold = 4
Ncval = 5
ml_strategy = ['train','validation']

#for rs in range(start_split,Nsplit):
for rs in range(start_split,start_split+1):
    
    # Creating a folder for each split
    split_savedir = os.path.join(trl_dir,'split'+str(rs))
    os.makedirs(split_savedir, exist_ok=True)
        
    ind = ind_testing[rs, :]
    shuffled_patient_classes = class_list_patients[ind]
    shuffled_patients = pat_ids_unique[ind]

    skf = StratifiedKFold(n_splits=2, shuffle=False) # shuffling is done earlier with permutations

    # Transcriptomic learning gets n_splits-1 folds (index1), classifier training gets 1 fold (index 2)
    # index1, index2 = next(iter(skf.split(ind, shuffled_patient_classes)))
    group1_index, group2_index = next(iter(skf.split(ind, shuffled_patient_classes)))

    print(f'Number of group 1 patients: {len(group1_index)}, Number of group 2 patients: {len(group2_index)}')

    pat_G1 = shuffled_patients[group1_index]
    pat_G2 = shuffled_patients[group2_index]
    pat_G1_classes = shuffled_patient_classes[group1_index]
    pat_G2_classes = shuffled_patient_classes[group2_index]

    # Make group 1 and group 2 datasets
    slide_ix_group1 = [ix for ix, d in enumerate(dataset.patients) if d in pat_G1]
    slide_ix_group2 = [ix for ix, d in enumerate(dataset.patients) if d in pat_G2]
    group1_dataset = subset_wsi_dataset(dataset,slide_ix_group1)
    
    group2_dataset = subset_wsi_dataset(dataset,slide_ix_group2)
    subtypes = list(group2_dataset.subtypes)

    print(f'Number of group 1 images: {len(group1_dataset)}, Number of group 2 images: {len(group2_dataset)}')
    

    for fold in range(start_fold,start_fold+1):
        
        #if rs==start_split and fold < start_fold:
        #    continue
        print(f'Running split {str(rs)}, fold {str(fold)}')

        ################################################################################ 
        # Obtain HE2Omics embeddings and make omic data
        ################################################################################ 
        model = torch.load(os.path.join('./' + exp.savedir,'split'+str(rs),'fold'+str(fold)+'_transcriptome_model.pt'),map_location='cpu')

        scores = []
        for count, sample in enumerate(group2_dataset.samples):
            #if count % 300 == 0:
            #    print(count)

            HE = np.load(sample[0])
            x = torch.Tensor(HE[np.newaxis].transpose(1, 2, 0))
            model.eval()
            with torch.no_grad():
                scores.append(model.conv_spare_last(x).detach().numpy().squeeze()) # Run partial inference

        # Each element in the 'scores' list object is able to inform about input features size
        model_params['input_feature_size'] = scores[0].shape[1]

        ### Define omic dataset
        omicdata = AlltilesOmic(scores,subtypes,class_names,scaleOmicFeatures=scaleOmicFeatures,n_tiles=n_tiles)

        ################################################################################
        # Creating CLAM folds (and a folder for each OMIC fold)
        ################################################################################
        fold_savedir = os.path.join(split_savedir,'fold'+str(fold))
        os.makedirs(fold_savedir, exist_ok=True)
        
        ## Stratified cross-validation in CLAM, stratify by subtype and trial
        # Different random seed for each OMIC fold to create CLAM folds
        #cval = fold   
        #train_idx, valid_idx, test_idx = patient_stratified_kfold(
        #                group2_dataset, n_splits=kfold_group2, valid_size=0.2,
        #                random_state=12*cval,stratifySubtype=False) # stratify set to False because it did not work when only 1/6 of the samples are used to train classifier

        ### Outfile name - base extensions
        out_overall = os.path.join(trl_dir,outfolder,outfile_base + f"_split_{rs}_fold_{fold}_overall.csv")
        out_auc = os.path.join(trl_dir,outfolder,outfile_base + f"_split_{rs}_fold_{fold}_class_AUC.csv")
        out_acc = os.path.join(trl_dir,outfolder,outfile_base + f"_split_{rs}_fold_{fold}_class_ACC.csv")
        # Text file
        outname = os.path.join(trl_dir,outfolder,outfile_base + f"_split_{rs}_fold_{fold}_output.txt")

        ov_metric = np.zeros((Ncval,2 * len(ml_strategy)))
        auc_collect = np.zeros((len(ml_strategy) * Ncval,len(class_names)))
        acc_collect = np.zeros((len(ml_strategy) * Ncval,len(class_names)))
        

        for cval in range(Ncval):  
            print(f'split: {rs}, OMIC fold:{fold}, CLAM fold: {cval}',file=open(outname, 'a'))
            print(f'split: {rs}, OMIC fold:{fold}, CLAM fold: {cval}')
            
            train_idx, valid_idx = patient_stratified_single_run(group2_dataset, valid_size=0.2, 
                                                                 stratifySubtype=True,random_state=12*cval)

            #################### save current folds
            this = (group2_dataset.patients[train_idx],group2_dataset.patients[valid_idx])
            with open(os.path.join(fold_savedir, 'group2_fold_patient_ids_TRL_clamFold' + str(cval) + '.pkl'), 'wb') as handle:
                pkl.dump(this, handle)
            ################################################################################ 

            # Separate train, validation, and test sets  
            train_set = Subset(omicdata, train_idx)
            valid_set = Subset(omicdata, valid_idx)

            ##### Omics-based classifier
            omic_model = OmicToSubtype(**model_params) # attention model
            omic_optimizer = omic_setup._setup_optimization(omic_model)
            #scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=4,gamma=0.9)
            omic_scheduler = omic_setup._setup_scheduler(omic_optimizer)

            # Train
            print("Training omics-based classifier")
            metric, class_auc, class_acc = omic_fit(omic_model,train_set,valid_set,
                               params=training_params,optimizer=omic_optimizer,
                               scheduler=omic_scheduler,
                               path=os.path.join(fold_savedir,'CLAM_fold_'+str(cval)),
                               outname = outname)
            
            ov_metric[cval] = metric.values
            # Allow printing output at each inner loop iteration
            ind_name = ['omicfold' + str(fold) + '_clamfold' + str(jnk) for jnk in range(Ncval)]
            ov_metric_df = pd.DataFrame(ov_metric,columns = ['tr_auc','tr_acc','val_auc','val_acc'],
                            index = ind_name)
            ov_metric_df.to_csv(out_overall,index=True)

            auc_collect[( len(ml_strategy)*cval):( len(ml_strategy) *(cval+1))] = class_auc
            # Allow printing output at each inner loop iteration
            auc_df = pd.DataFrame(auc_collect, columns = ['class1','class2','class3','class4'],
                        index = np.tile(['train','val'],Ncval))

            acc_collect[( len(ml_strategy)*cval):( len(ml_strategy) *(cval+1))] = class_acc
            # Allow printing output at each inner loop iteration
            acc_df = pd.DataFrame(acc_collect, columns = ['class1','class2','class3','class4'],
                        index = np.tile(['train','val'],Ncval))

            auc_df.to_csv(out_auc,index=True)
            acc_df.to_csv(out_acc,index=True)